<a href="https://colab.research.google.com/github/farhatsattar/ATM_PROJ/blob/main/assignment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Installation
!pip install --upgrade --quiet google-genai


In [5]:
!pip install gTTS --quiet

In [6]:
!pip install playsound --quiet

  Preparing metadata (setup.py) ... done


In [7]:
# API Key setup
from google.colab import userdata
GEMINI_API_KEY: str = userdata.get('GOOGLE_API_KEY_3')
if(GEMINI_API_KEY):
  print("API Key found")
else:
  print("API Key not found")

API Key found


In [8]:
# Initilize and configure the client
# Select the model
from google import genai
from google.genai import Client

client: Client = genai.Client(
    api_key=GEMINI_API_KEY,
)

model: str = "gemini-2.0-flash-exp"

In [9]:
video_file_name = "/content/my_video.mp4"

In [10]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload(path=video_file_name)
  while video_file.state == "PROCESSING":
      print('Waiting for video to be processed.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name or "")

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + (video_file.uri or ""))

  return video_file

my_video = upload_video('my_video.mp4')

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/2ihp58de5pbg


In [14]:
from google.genai.types import Content, Part
from IPython.display import display, Markdown
prompt = """For each scene in this video,
            generate captions that describe the scene along with any spoken text placed in quotation marks.
            Place each caption into an object with the timecode of the caption in the video.
         """

video = my_video

response = client.models.generate_content(
    model=model,
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video.uri or "",
                    mime_type=video.mime_type or ""),
                ]),
        prompt,
    ]
)

Markdown(response.text)


```json
[
  {
    "timecode": "00:00",
    "caption": "A laptop with news articles visible on the screen. The laptop keyboard and a portion of a dark blue desk surface are also visible.  A voice states, \"Assalamualaikum. My name is Farah Zafar and I am student of P I A I C. Where I am learning about AI.\""
  }
]
```

In [16]:
from gtts import gTTS
from playsound import playsound
from IPython.display import Audio
gtts = gTTS(text= "/asalm-o-alikum, my name is farhat zafar and i am student of PIAIC where i am learning AI", lang='en', slow=False)
with open("output.mp4", "wb") as f:
  for chunk in gtts.stream():
    f.write(chunk)
display(Audio("output.mp4",autoplay=True))